In [1]:
from bs4 import BeautifulSoup
from random import randint, shuffle
from time import sleep
import re
import pandas as pd
import json
import math
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
from tqdm import tqdm_notebook as tqdm
import shutil
import numpy as np
import pandas_profiling


pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", -1)
pd.set_option("display.float_format", "{:,.2f}".format)
pd.set_option("display.max_info_rows", 200)

In [2]:
import sephora_scrape as scrape
%reload_ext autoreload
%autoreload 2

In [3]:
reviews_folder = "/Users/valmadrid/DataScienceBootcamp/Projects/Gift Recommendation/Gift-Recommendation/reviews/"
if not os.path.exists(reviews_folder):
    print("check file path")

In [4]:
downloaded_json_stats = [
    file for file in os.listdir(reviews_folder) if file.endswith("_stats.json")
]
downloaded_json_stats[:10]

['P163103_stats.json',
 'P12045_stats.json',
 'P386462_stats.json',
 'P425604_stats.json',
 'P398009_stats.json',
 'P416103_stats.json',
 'P151107_stats.json',
 'P454806_stats.json',
 'P448444_stats.json',
 'P417162_stats.json']

In [5]:
def get_reviewstats(filename, folder_path):
    
    with open(folder_path + filename) as json_file:
        reviews_data = json.load(json_file)
    
    reviews = []
    
    product_id = filename.replace("_stats.json", "")
    
    if reviews_data["TotalResults"] == 0 or reviews_data["HasErrors"] == True:
        return reviews
        
    else:
        for key, val in reviews_data["Includes"]["Products"].items():
            product_review = {}
            product_review["id"] = product_id
            for key2, val2 in val.items():
                if key2 == "ReviewStatistics":
                    for key3, val3, in val["ReviewStatistics"].items():
                        product_review[key3] = val3
            reviews.append(product_review)
            if len(reviews) == 1:
                return reviews

In [6]:
df_reviewstats = pd.DataFrame()
no_stats=[]

for filename in tqdm(downloaded_json_stats):
    reviews = get_reviewstats(filename, reviews_folder)
    if len(reviews)==0:
        no_stats.append(filename)
    else:
        df_reviewstats = df_reviewstats.append(pd.DataFrame(reviews),
                                                   ignore_index=True)
        
df_reviewstats.to_csv("reviewstats.csv", index=False)

In [7]:
for i in tqdm(range(len(df_reviewstats))):
    if df_reviewstats["RatingDistribution"].iloc[i]:
        for rating in df_reviewstats["RatingDistribution"].iloc[i]:
            col_name = (re.sub(
                r"[^a-zA-Z0-9]+", " ",
                ("rating_" + str(rating["RatingValue"]) + "_count"))).replace(
                    " ", "_")
            df_reviewstats.at[i, col_name] = rating["Count"]
            df_reviewstats[col_name] = df_reviewstats[col_name].astype(
                "int64", errors="ignore")

    if df_reviewstats["ContextDataDistribution"].iloc[i]:
        for context, value in df_reviewstats["ContextDataDistribution"].iloc[
                i].items():
            for context2, value2 in value.items():
                if context2 == "Values":
                    for value3 in value2:
                        col_name = (re.sub(
                            r"[^a-zA-Z0-9]+", " ",
                            (context.lower() + "_" + value3["Value"].lower() +
                             "_count"))).replace(" ", "_")
                        df_reviewstats.at[i, col_name] = value3["Count"]
                        df_reviewstats[col_name] = df_reviewstats[
                            col_name].astype("int64", errors="ignore")

#     if df_reviewstats["TagDistribution"].iloc[i]:
#         for tag, value in df_reviewstats["TagDistribution"].iloc[i].items():
#             for tag2, value2 in value.items():
#                 if tag2 == "Values":
#                     for value3 in value2:
#                         col_name = (re.sub(
#                             r"[^a-zA-Z0-9]+", " ",
#                             (tag.lower() + "_" + value3["Value"].lower() +
#                              "_count"))).replace(" ", "_")
#                         df_reviewstats.at[i, col_name] = value3["Count"]
#                         df_reviewstats[col_name] = df_reviewstats[
#                             col_name].astype("int64", errors="ignore")

df_reviewstats.to_csv("reviewstats2.csv", index=False)

In [8]:
df_reviewstats.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8789 entries, 0 to 8788
Data columns (total 95 columns):
id                                           8789 non-null object
FeaturedReviewCount                          8789 non-null int64
RecommendedCount                             8789 non-null int64
SecondaryRatingsAveragesOrder                8789 non-null object
AverageOverallRating                         8789 non-null float64
NotHelpfulVoteCount                          8789 non-null int64
RatingDistribution                           8789 non-null object
RatingsOnlyReviewCount                       8789 non-null int64
FirstSubmissionTime                          8789 non-null object
TagDistribution                              8789 non-null object
ContextDataDistribution                      8789 non-null object
TotalReviewCount                             8789 non-null int64
LastSubmissionTime                           8789 non-null object
TagDistributionOrder                   

In [9]:
df_reviewstats.columns[19:]

Index(['rating_1_count', 'rating_2_count', 'rating_3_count', 'rating_4_count',
       'rating_5_count', 'skintype_normal_count', 'skintype_combination_count',
       'skintype_dry_count', 'skintype_oily_count', 'eyecolor_blue_count',
       'eyecolor_brown_count', 'eyecolor_green_count', 'eyecolor_hazel_count',
       'eyecolor_gray_count', 'staffcontext_true_count',
       'staffcontext_false_count', 'haircolor_blonde_count',
       'haircolor_brunette_count', 'haircolor_auburn_count',
       'haircolor_black_count', 'haircolor_red_count', 'haircolor_gray_count',
       'skintone_fair_count', 'skintone_light_count', 'skintone_medium_count',
       'skintone_olive_count', 'skintone_deep_count', 'skintone_dark_count',
       'skintone_porcelain_count', 'skintone_tan_count',
       'skintone_ebony_count', 'incentivizedreview_true_count',
       'incentivizedreview_false_count', 'age_13to17_count',
       'age_18to24_count', 'age_25to34_count', 'age_35to44_count',
       'age_45to54_count

In [10]:
for col in df_reviewstats.columns[19:]:
    df_reviewstats[col].fillna(0, inplace=True)

In [11]:
sum(df_reviewstats.isna().sum())

0

In [12]:
df_reviewstats.to_csv("reviewstats2.csv", index=False)

In [13]:
df_reviewstats.columns[:19]

Index(['id', 'FeaturedReviewCount', 'RecommendedCount',
       'SecondaryRatingsAveragesOrder', 'AverageOverallRating',
       'NotHelpfulVoteCount', 'RatingDistribution', 'RatingsOnlyReviewCount',
       'FirstSubmissionTime', 'TagDistribution', 'ContextDataDistribution',
       'TotalReviewCount', 'LastSubmissionTime', 'TagDistributionOrder',
       'OverallRatingRange', 'NotRecommendedCount', 'HelpfulVoteCount',
       'SecondaryRatingsAverages', 'ContextDataDistributionOrder'],
      dtype='object')

In [14]:
df_report = df_reviewstats.iloc[:,:19].profile_report(title="Sephora Product Review Stats", correlation_threshold_pearson = .9, sort = "None")
df_report

In [15]:
to_exclude = [
    "FeaturedReviewCount", "SecondaryRatingsAveragesOrder",
    "NotHelpfulVoteCount", "RatingDistribution", "FirstSubmissionTime",
    "ContextDataDistribution", "RatingsOnlyReviewCount",
    "TagDistributionOrder", "LastSubmissionTime", "TagDistributionOrder",
    "HelpfulVoteCount", "SecondaryRatingsAverages", "OverallRatingRange",
    "ContextDataDistributionOrder", "TagDistribution"
]

df_stats = df_reviewstats.drop(columns=to_exclude)
df_stats

,id,RecommendedCount,AverageOverallRating,TotalReviewCount,NotRecommendedCount,rating_1_count,rating_2_count,rating_3_count,rating_4_count,rating_5_count,skintype_normal_count,skintype_combination_count,skintype_dry_count,skintype_oily_count,eyecolor_blue_count,eyecolor_brown_count,eyecolor_green_count,eyecolor_hazel_count,eyecolor_gray_count,staffcontext_true_count,staffcontext_false_count,haircolor_blonde_count,haircolor_brunette_count,haircolor_auburn_count,haircolor_black_count,haircolor_red_count,haircolor_gray_count,skintone_fair_count,skintone_light_count,skintone_medium_count,skintone_olive_count,skintone_deep_count,skintone_dark_count,skintone_porcelain_count,skintone_tan_count,skintone_ebony_count,incentivizedreview_true_count,incentivizedreview_false_count,age_13to17_count,age_18to24_count,age_25to34_count,age_35to44_count,age_45to54_count,age_over54_count,skinconcerns_acne_count,skinconcerns_aging_count,skinconcerns_blackheads_count,skinconcerns_calluses_count,skinconcerns_cellulite_count,skinconcerns_cuticles_count,skinconcerns_darkcircles_count,skinconcerns_dullness_count,skinconcerns_pores_count,skinconcerns_redness_count,skinconcerns_sensitivity_count,skinconcerns_stretchmarks_count,skinconcerns_sundamage_count,skinconcerns_unevenskintones_count,hairconcerns_antiaging_count,hairconcerns_colorprotection_count,hairconcerns_curlyenhancing_count,hairconcerns_damage_count,hairconcerns_dandruff_count,hairconcerns_frizz_count,hairconcerns_heatprotection_count,hairconcerns_hold_count,hairconcerns_oiliness_count,hairconcerns_shine_count,hairconcerns_straighteningsmoothing_count,hairconcerns_volumizing_count,haircondition_normal_count,haircondition_oily_count,haircondition_dry_count,haircondition_fine_count,haircondition_coarse_count,haircondition_curly_count,haircondition_chemicallytreated_count,haircondition_straight_count,hairconcerns_thinning_count,haircondition_wavy_count,skinconcerns_puffiness_count
0,P163103,375,4.56,1858,34,42.00,52.00,101.00,294.00,"1,369.00",53.00,215.00,65.00,50.00,75.00,211.00,44.00,45.00,2.00,1.00,523.00,69.00,188.00,17.00,94.00,11.00,4.00,71.00,120.00,82.00,22.00,18.00,16.00,22.00,28.00,6.00,64.00,460.00,41.00,368.00,396.00,141.00,94.00,44.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,P12045,590,4.44,1348,85,46.00,46.00,88.00,261.00,907.00,79.00,476.00,176.00,86.00,115.00,370.00,66.00,75.00,4.00,8.00,548.00,112.00,304.00,16.00,182.00,21.00,6.00,207.00,215.00,157.00,56.00,28.00,14.00,48.00,57.00,2.00,3.00,553.00,4.00,59.00,109.00,54.00,26.00,24.00,297.00,123.00,44.00,1.00,4.00,4.00,41.00,21.00,2.00,23.00,25.00,1.00,5.00,4.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,P386462,111,4.12,348,34,30.00,25.00,28.00,55.00,210.00,18.00,76.00,28.00,11.00,29.00,51.00,27.00,20.00,3.00,1.00,179.00,37.00,67.00,9.00,16.00,3.00,4.00,41.00,31.00,34.00,4.00,3.00,2.00,11.00,7.00,0.00,17.00,163.00,2.00,19.00,55.00,28.00,17.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,24.00,25.00,7.00,35.00,7.00,18.00,3.00,1.00,1.00,3.00,1.00,3.00,18.00,4.00,13.00,26.00,9.00,17.00,34.00,1.00,0.00,0.00,0.00
3,P425604,35,4.20,46,11,2.00,3.00,4.00,12.00,25.00,4.00,26.00,8.00,8.00,5.00,25.00,7.00,7.00,1.00,0.00,18.00,12.00,22.00,4.00,7.00,1.00,0.00,14.00,15.00,5.00,2.00,1.00,0.00,4.00,5.00,0.00,0.00,18.00,1.00,1.00,1.00,3.00,2.00,4.00,10.00,7.00,3.00,0.00,0.00,0.00,3.00,1.00,0.00,1.00,0.00,0.00,0.00,1.00,2.00,6.00,2.00,0.00,2.00,5.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,2.00,0.00,1.00,0.00,0.00
4,P398009,1053,3.99,2748,220,169.00,231.00,338.00,727.00,"1,283.00",261.00,896.00,288.00,306.00,365.00,933.00,211.00,190.00,20.00,10.00,"1,767.00",259.00,578.00,52.00,274.00,47.00,10.00,376.00,402.00,362.00,145.00,130.00,75.00,161.00,94.00,19.00,6.00,"1,771.00",67.00,225.0

In [16]:
for col in df_stats.columns[1:]:
    if col !="AverageOverallRating":
        df_stats[col] = df_stats[col].astype("int64")

In [17]:
df_stats.head()

,id,RecommendedCount,AverageOverallRating,TotalReviewCount,NotRecommendedCount,rating_1_count,rating_2_count,rating_3_count,rating_4_count,rating_5_count,skintype_normal_count,skintype_combination_count,skintype_dry_count,skintype_oily_count,eyecolor_blue_count,eyecolor_brown_count,eyecolor_green_count,eyecolor_hazel_count,eyecolor_gray_count,staffcontext_true_count,staffcontext_false_count,haircolor_blonde_count,haircolor_brunette_count,haircolor_auburn_count,haircolor_black_count,haircolor_red_count,haircolor_gray_count,skintone_fair_count,skintone_light_count,skintone_medium_count,skintone_olive_count,skintone_deep_count,skintone_dark_count,skintone_porcelain_count,skintone_tan_count,skintone_ebony_count,incentivizedreview_true_count,incentivizedreview_false_count,age_13to17_count,age_18to24_count,age_25to34_count,age_35to44_count,age_45to54_count,age_over54_count,skinconcerns_acne_count,skinconcerns_aging_count,skinconcerns_blackheads_count,skinconcerns_calluses_count,skinconcerns_cellulite_count,skinconcerns_cuticles_count,skinconcerns_darkcircles_count,skinconcerns_dullness_count,skinconcerns_pores_count,skinconcerns_redness_count,skinconcerns_sensitivity_count,skinconcerns_stretchmarks_count,skinconcerns_sundamage_count,skinconcerns_unevenskintones_count,hairconcerns_antiaging_count,hairconcerns_colorprotection_count,hairconcerns_curlyenhancing_count,hairconcerns_damage_count,hairconcerns_dandruff_count,hairconcerns_frizz_count,hairconcerns_heatprotection_count,hairconcerns_hold_count,hairconcerns_oiliness_count,hairconcerns_shine_count,hairconcerns_straighteningsmoothing_count,hairconcerns_volumizing_count,haircondition_normal_count,haircondition_oily_count,haircondition_dry_count,haircondition_fine_count,haircondition_coarse_count,haircondition_curly_count,haircondition_chemicallytreated_count,haircondition_straight_count,hairconcerns_thinning_count,haircondition_wavy_count,skinconcerns_puffiness_count
0,P163103,375,4.56,1858,34,42,52,101,294,1369,53,215,65,50,75,211,44,45,2,1,523,69,188,17,94,11,4,71,120,82,22,18,16,22,28,6,64,460,41,368,396,141,94,44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,P12045,590,4.44,1348,85,46,46,88,261,907,79,476,176,86,115,370,66,75,4,8,548,112,304,16,182,21,6,207,215,157,56,28,14,48,57,2,3,553,4,59,109,54,26,24,297,123,44,1,4,4,41,21,2,23,25,1,5,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,P386462,111,4.12,348,34,30,25,28,55,210,18,76,28,11,29,51,27,20,3,1,179,37,67,9,16,3,4,41,31,34,4,3,2,11,7,0,17,163,2,19,55,28,17,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,25,7,35,7,18,3,1,1,3,1,3,18,4,13,26,9,17,34,1,0,0,0
3,P425604,35,4.20,46,11,2,3,4,12,25,4,26,8,8,5,25,7,7,1,0,18,12,22,4,7,1,0,14,15,5,2,1,0,4,5,0,0,18,1,1,1,3,2,4,10,7,3,0,0,0,3,1,0,1,0,0,0,1,2,6,2,0,2,5,1,0,0,0,0,0,0,0,1,0,0,0,2,0,1,0,0
4,P398009,1053,3.99,2748,220,169,231,338,727,1283,261,896,288,306,365,933,211,190,20,10,1767,259,578,52,274,47,10,376,402,362,145,130,75,161,94,19,6,1771,67,225,231,55,35,42,578,181,110,2,9,3,135,30,4,31,23,2,7,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
df_stats.rename(
    {
        "RecommendedCount": "recommended_count",
        "AverageOverallRating": "average_rating",
        "TotalReviewCount": "review_count",
        "NotRecommendedCount": "not_recommended_count"
    },
    axis=1,
    inplace=True)

In [39]:
len(df_stats.columns), df_stats.columns

(81, Index(['id', 'recommended_count', 'average_rating', 'review_count',
        'not_recommended_count', 'rating_1_count', 'rating_2_count',
        'rating_3_count', 'rating_4_count', 'rating_5_count',
        'skintype_normal_count', 'skintype_combination_count',
        'skintype_dry_count', 'skintype_oily_count', 'eyecolor_blue_count',
        'eyecolor_brown_count', 'eyecolor_green_count', 'eyecolor_hazel_count',
        'eyecolor_gray_count', 'staffcontext_true_count',
        'staffcontext_false_count', 'haircolor_blonde_count',
        'haircolor_brunette_count', 'haircolor_auburn_count',
        'haircolor_black_count', 'haircolor_red_count', 'haircolor_gray_count',
        'skintone_fair_count', 'skintone_light_count', 'skintone_medium_count',
        'skintone_olive_count', 'skintone_deep_count', 'skintone_dark_count',
        'skintone_porcelain_count', 'skintone_tan_count',
        'skintone_ebony_count', 'incentivizedreview_true_count',
        'incentivizedreview_false_

In [20]:
rating = [col for col in df_stats.columns if "rating" in col]
rating

['average_rating',
 'rating_1_count',
 'rating_2_count',
 'rating_3_count',
 'rating_4_count',
 'rating_5_count']

In [21]:
age = [col for col in df_stats.columns if col.startswith("age")]
age

['age_13to17_count',
 'age_18to24_count',
 'age_25to34_count',
 'age_35to44_count',
 'age_45to54_count',
 'age_over54_count']

In [31]:
skintone = [col for col in df_stats.columns if col.startswith("skintone")]
skintone

['skintone_fair_count',
 'skintone_light_count',
 'skintone_medium_count',
 'skintone_olive_count',
 'skintone_deep_count',
 'skintone_dark_count',
 'skintone_porcelain_count',
 'skintone_tan_count',
 'skintone_ebony_count']

In [32]:
skinconcerns = [col for col in df_stats.columns if col.startswith("skinconcerns")]
skinconcerns

['skinconcerns_acne_count',
 'skinconcerns_aging_count',
 'skinconcerns_blackheads_count',
 'skinconcerns_calluses_count',
 'skinconcerns_cellulite_count',
 'skinconcerns_cuticles_count',
 'skinconcerns_darkcircles_count',
 'skinconcerns_dullness_count',
 'skinconcerns_pores_count',
 'skinconcerns_redness_count',
 'skinconcerns_sensitivity_count',
 'skinconcerns_stretchmarks_count',
 'skinconcerns_sundamage_count',
 'skinconcerns_unevenskintones_count',
 'skinconcerns_puffiness_count']

In [33]:
skintype = [col for col in df_stats.columns if col.startswith("skintype")]
skintype

['skintype_normal_count',
 'skintype_combination_count',
 'skintype_dry_count',
 'skintype_oily_count']

In [34]:
eyecolor = [col for col in df_stats.columns if col.startswith("eyecolor") ]
eyecolor

['eyecolor_blue_count',
 'eyecolor_brown_count',
 'eyecolor_green_count',
 'eyecolor_hazel_count',
 'eyecolor_gray_count']

In [35]:
haircolor = [col for col in df_stats.columns if col.startswith("haircolor")]
haircolor

['haircolor_blonde_count',
 'haircolor_brunette_count',
 'haircolor_auburn_count',
 'haircolor_black_count',
 'haircolor_red_count',
 'haircolor_gray_count']

In [36]:
haircondition = [col for col in df_stats.columns if col.startswith("haircondition")]
haircondition

['haircondition_normal_count',
 'haircondition_oily_count',
 'haircondition_dry_count',
 'haircondition_fine_count',
 'haircondition_coarse_count',
 'haircondition_curly_count',
 'haircondition_chemicallytreated_count',
 'haircondition_straight_count',
 'haircondition_wavy_count']

In [37]:
hairconcerns = [col for col in df_stats.columns if col.startswith("hairconcerns")]
hairconcerns

['hairconcerns_antiaging_count',
 'hairconcerns_colorprotection_count',
 'hairconcerns_curlyenhancing_count',
 'hairconcerns_damage_count',
 'hairconcerns_dandruff_count',
 'hairconcerns_frizz_count',
 'hairconcerns_heatprotection_count',
 'hairconcerns_hold_count',
 'hairconcerns_oiliness_count',
 'hairconcerns_shine_count',
 'hairconcerns_straighteningsmoothing_count',
 'hairconcerns_volumizing_count',
 'hairconcerns_thinning_count']

In [38]:
others = [
    "staffcontext_true_count", "staffcontext_false_count",
    "incentivizedreview_true_count", "incentivizedreview_true_count"
]

In [40]:
df_stats = df_stats[[
    "id", "review_count", "recommended_count", "not_recommended_count"
]+rating+age+skintone+skintype+skinconcerns+eyecolor+haircolor+haircondition+hairconcerns+others].copy()
df_stats

,id,review_count,recommended_count,not_recommended_count,average_rating,rating_1_count,rating_2_count,rating_3_count,rating_4_count,rating_5_count,age_13to17_count,age_18to24_count,age_25to34_count,age_35to44_count,age_45to54_count,age_over54_count,skintone_fair_count,skintone_light_count,skintone_medium_count,skintone_olive_count,skintone_deep_count,skintone_dark_count,skintone_porcelain_count,skintone_tan_count,skintone_ebony_count,skintype_normal_count,skintype_combination_count,skintype_dry_count,skintype_oily_count,skinconcerns_acne_count,skinconcerns_aging_count,skinconcerns_blackheads_count,skinconcerns_calluses_count,skinconcerns_cellulite_count,skinconcerns_cuticles_count,skinconcerns_darkcircles_count,skinconcerns_dullness_count,skinconcerns_pores_count,skinconcerns_redness_count,skinconcerns_sensitivity_count,skinconcerns_stretchmarks_count,skinconcerns_sundamage_count,skinconcerns_unevenskintones_count,skinconcerns_puffiness_count,eyecolor_blue_count,eyecolor_brown_count,eyecolor_green_count,eyecolor_hazel_count,eyecolor_gray_count,haircolor_blonde_count,haircolor_brunette_count,haircolor_auburn_count,haircolor_black_count,haircolor_red_count,haircolor_gray_count,haircondition_normal_count,haircondition_oily_count,haircondition_dry_count,haircondition_fine_count,haircondition_coarse_count,haircondition_curly_count,haircondition_chemicallytreated_count,haircondition_straight_count,haircondition_wavy_count,hairconcerns_antiaging_count,hairconcerns_colorprotection_count,hairconcerns_curlyenhancing_count,hairconcerns_damage_count,hairconcerns_dandruff_count,hairconcerns_frizz_count,hairconcerns_heatprotection_count,hairconcerns_hold_count,hairconcerns_oiliness_count,hairconcerns_shine_count,hairconcerns_straighteningsmoothing_count,hairconcerns_volumizing_count,hairconcerns_thinning_count,staffcontext_true_count,staffcontext_false_count,incentivizedreview_true_count,incentivizedreview_true_count
0,P163103,1858,375,34,4.56,42,52,101,294,1369,41,368,396,141,94,44,71,120,82,22,18,16,22,28,6,53,215,65,50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,75,211,44,45,2,69,188,17,94,11,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,523,64,64
1,P12045,1348,590,85,4.44,46,46,88,261,907,4,59,109,54,26,24,207,215,157,56,28,14,48,57,2,79,476,176,86,297,123,44,1,4,4,41,21,2,23,25,1,5,4,0,115,370,66,75,4,112,304,16,182,21,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,548,3,3
2,P386462,348,111,34,4.12,30,25,28,55,210,2,19,55,28,17,10,41,31,34,4,3,2,11,7,0,18,76,28,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,29,51,27,20,3,37,67,9,16,3,4,18,4,13,26,9,17,34,1,0,24,25,7,35,7,18,3,1,1,3,1,3,0,1,179,17,17
3,P425604,46,35,11,4.20,2,3,4,12,25,1,1,1,3,2,4,14,15,5,2,1,0,4,5,0,4,26,8,8,10,7,3,0,0,0,3,1,0,1,0,0,0,1,0,5,25,7,7,1,12,22,4,7,1,0,0,0,1,0,0,0,2,0,0,2,6,2,0,2,5,1,0,0,0,0,0,1,0,18,0,0
4,P398009,2748,1053,220,3.99,169,231,338,727,1283,67,225,231,55,35,42,376,402,362,145,130,75,161,94,19,261,896,288,306,578,181,110,2,9,3,135,30,4,31,23,2,7,19,0,365,933,211,190,20,259,578,52,274,47,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,1767,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8784,P456594,21,13,8,3.43,4,3,1,6,7,0,1,2,0,0,2,2,9,3,1,3,0,3,0,0,3,8,5,5,10,4,1,0,0,0,2,2,0,0,0,0,1,0,0,4,12,4,1,0,3,11,2,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,20,0,0
8785,P427718,261,160,101,3.61,54,30,19,20,138,1,10,17,6,3,4,58,76,60,16,10,3,15,12,0,36,145,37,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,45,128,31,41,3,66,120,5,55,6,1,4,1,14,7,5,3,35,0,5,35,55,9,72,13,23,3,1,6,9,4,9,3,1,119,4,4
8786,P459305,29,28,1,4.76,0,0,1,5,23,1,2,0,0,0,0,8,10,4,1,0,0,3,2,0,5,14,1,8,22,3,2,0,0,0,0,0,0,0,0,0,0,0,0,7,9,5,6,0,12,11,1,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,29,28,28
8787,P428818,22,15,7,4.00,1

In [41]:
df_stats.to_csv("reviewstats2.csv", index=False)